<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/U-net_dataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 28.3 MB/s eta 0:00:00


In [3]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode

# Additional code can be added here if needed

In [ ]:
# image_paths_2018 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2018/*.tif")
# image_paths_2019 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2019/*.tif")
# image_paths_2020 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2020/*.tif")
# dem_path = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/dem/*.tif")
# nwi_ccap_path = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/nwi_ccap_filled/*.tif")
# label_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/ccap_filled/*.tif")

train test split

In [42]:
planet_images = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images/*.tif")

In [ ]:
planet_images

In [50]:
planet_labels = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/*.tif")

In [ ]:
planet_labels

In [44]:
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [62]:
# prompt: define a function to extract the numeric identifier from a file path to include all information before the second "_". should be something like this: 4444_2452

def extract_id(path):
  filename = os.path.basename(path)
  parts = filename.split('_')
  return parts[0] + '_' + parts[1]


In [10]:
# def extract_id(file_path):
#     # This function extracts the numeric identifier from a file path, assuming the format you provided
#     return int(os.path.basename(file_path).split('_')[0])

In [65]:
# Create a unified list of identifiers from your labeled dataset, assuming nwi_ccap_path holds all possible labels
identifiers = [extract_id(path) for path in planet_images]

In [66]:
identifiers

['6061_2364',
 '6015_2361',
 '6111_2359',
 '6053_2361',
 '6103_2365',
 '6103_2364',
 '4671_2441',
 '4656_2441',
 '6068_2362',
 '6049_2361',
 '6003_2360',
 '6084_2359',
 '4648_2439',
 '6003_2363',
 '6076_2362',
 '6045_2362',
 '6022_2366',
 '6064_2360',
 '6007_2366',
 '6011_2363',
 '6049_2366',
 '4671_2440',
 '7597_2314',
 '4656_2442',
 '6015_2360',
 '6099_2363',
 '7601_2315',
 '6087_2365',
 '6064_2362',
 '6018_2363',
 '6026_2363',
 '5995_2363',
 '6038_2364',
 '6061_2361',
 '4675_2442',
 '6111_2366',
 '7609_2314',
 '6030_2362',
 '6038_2363',
 '6011_2361',
 '6007_2360',
 '7605_2315',
 '4660_2442',
 '5995_2359',
 '4668_2442',
 '5999_2361',
 '7616_2314',
 '6003_2365',
 '7609_2315',
 '6095_2366',
 '7586_2313',
 '6003_2361',
 '6026_2360',
 '7593_2313',
 '6003_2359',
 '6084_2364',
 '6018_2366',
 '6053_2360',
 '6007_2361',
 '6099_2362',
 '6068_2366',
 '6041_2363',
 '6061_2359',
 '6080_2363',
 '6041_2361',
 '6087_2366',
 '6045_2359',
 '6022_2362',
 '6030_2366',
 '6038_2359',
 '7601_2314',
 '4664

In [67]:
# Split identifiers into training, validation, and test sets
train_ids, temp_ids = train_test_split(identifiers, test_size=0.3, random_state=42)
validation_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

In [68]:
# Function to split paths based on their identifiers
def split_paths(paths, ids_set):
    return [path for path in paths if extract_id(path) in ids_set]

In [71]:
planet_labels_path

['/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/6053_2362_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/7627_2301_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/7018_2334_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/7596_2302_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/6080_2362_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/6293_2367_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/6984_2335_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/labels/6068_2362_nwi_ccap_filled.tif',
 '/content/drive/MyDrive/DeepLea

In [ ]:
# # Now, split all your datasets according to these ids
# train_image_paths_2018 = split_paths(image_paths_2018, set(train_ids))
# validation_image_paths_2018 = split_paths(image_paths_2018, set(validation_ids))
# test_image_paths_2018 = split_paths(image_paths_2018, set(test_ids))

In [ ]:

# Repeat the above step for image_paths_2019, image_paths_2020, dem_path, nwi_ccap_path, etc.

# Now you have all your datasets split into training, validation, and test sets by their paths
# The next steps involve loading these paths into your model for training, which can be done using data generators or similar methods

In [ ]:
# # Now, split all your datasets according to these ids
# train_image_paths_2019 = split_paths(image_paths_2019, set(train_ids))
# validation_image_paths_2019 = split_paths(image_paths_2019, set(validation_ids))
# test_image_paths_2019 = split_paths(image_paths_2019, set(test_ids))

In [ ]:
# # Now, split all your datasets according to these ids
# train_image_paths_2020 = split_paths(image_paths_2020, set(train_ids))
# validation_image_paths_2020 = split_paths(image_paths_2020, set(validation_ids))
# test_image_paths_2020 = split_paths(image_paths_2020, set(test_ids))

In [ ]:
# # Now, split all your datasets according to these ids
# train_dem_path = split_paths(dem_path, set(train_ids))
# validation_dem_path = split_paths(dem_path, set(validation_ids))
# test_dem_path = split_paths(dem_path, set(test_ids))

In [ ]:
# # Now, split all your datasets according to these ids
# train_nwi_ccap_path = split_paths(nwi_ccap_path, set(train_ids))
# validation_nwi_ccap_path = split_paths(nwi_ccap_path, set(validation_ids))
# test_nwi_ccap_path = split_paths(nwi_ccap_path, set(test_ids))

In [69]:
# Now, split all your datasets according to these ids
planet_images_path = split_paths(planet_images, set(train_ids))
planet_labels_path = split_paths(planet_labels, set(train_ids))

end

In [73]:
class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_classes = num_classes
        # self.noise = noise

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        image_batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        label_batch_files = self.label_files[index * self.batch_size : (index + 1) * self.batch_size]

        batch_images, batch_labels = self.load_images_and_labels(image_batch_files,label_batch_files)
        return batch_images, batch_labels

    def load_and_reshape_image(self, image_path):
        with rasterio.open(image_path) as src:
            image = src.read()
            image = image.transpose((1, 2, 0))
            if image.shape[0] != self.img_height or image.shape[1] != self.img_width:
                image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)
            if len(image.shape) == 3 and image.shape[2] == 1:
                image = np.squeeze(image, axis=2)

            return image

    def load_images_and_labels(self, image_files, label_files):
        images = []
        labels = []

        for image_file in image_files:
            image = self.load_and_reshape_image(image_file)
            # image[image <= -3e+38] = np.nan

            # # # Replace NaN values with the mean of the non-NaN pixels
            # if np.any(np.isnan(image)):
            #     nan_mask = np.isnan(image)
            #     image[nan_mask] = np.nanmean(image)

            # # # Replace Inf values with the mean of the non-Inf pixels
            # if np.any(np.isinf(image)):
            #     inf_mask = np.isinf(image)
            #     image[inf_mask] = np.nanmean(image)

            # # Convert to float
            # image = image.astype(np.float32)

            # # Z-score normalization
            # mean = np.mean(image, axis=(0, 1), keepdims=True)
            # std = np.std(image, axis=(0, 1), keepdims=True)
            # # mean[mean < 0]
            # std[std < 0] = 0

            # # Normalize with epsilon to prevent divide by zero
            # epsilon = 1e-7

            # image = (image - mean) / (std + epsilon)

            images.append(image)

        for label_file in label_files:
            # label_file = image_file.replace("planet", "ccap_filled").replace('images_2018','ccap_filled')
            label = self.load_and_reshape_image(label_file)
            # label[label <= -3e+38] = np.nan

            # # # Replace NaN values with the mean of the non-NaN pixels
            # if np.any(np.isnan(label)):
            #     nan_mask = np.isnan(label)
            #     label[nan_mask] = 12

            # # # Replace Inf values with the mean of the non-Inf pixels
            # if np.any(np.isinf(label)):
            #     inf_mask = np.isinf(label)
            #     label[inf_mask] = 12

            label -= 1
            label = label.astype(np.int8)
            label = to_categorical(label, num_classes=self.num_classes)
            labels.append(label)

        return np.array(images), np.array(labels)

In [ ]:
# train_image_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/images_2018/*.tif")
# train_label_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ccap_filled/*.tif")

# # val_image_paths = glob.glob(f"{base_dir}/validation/image/*.tif")
# # val_label_paths = glob.glob(f"{base_dir}/validation/label/*.tif")


planet_images_path = split_paths(planet_images, set(train_ids))
planet_labels_path = split_paths(planet_labels, set(train_ids))

In [ ]:
# /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/images_2018

In [74]:
# data_gen_test = DataGenerator(sum_df['Images_path'][0:5], sum_df['Labels_path'][0:5], 512, 512, 2, 22)
# Create the data generator
training_data_generator = DataGenerator(planet_images_path, planet_labels_path, 128, 128, 4, 12)
# validation_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==1], sum_df['Labels_path'][sum_df['random_split']==1], 512, 512, 4, 25)


In [ ]:
# /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ccap_filled/1004_ccap_filled.tif

In [76]:
batch_images, batch_labels = training_data_generator.__getitem__(0)

In [77]:
batch_images

array([[[[ 2.1700000e+02,  3.9000000e+02,  3.8600000e+02, ...,
           3.3300000e+02,  2.8380000e+03,  1.0000000e+00],
         [ 2.1200000e+02,  3.8600000e+02,  3.8800000e+02, ...,
           3.3000000e+02,  2.8700000e+03,  1.0000000e+00],
         [ 2.1800000e+02,  3.9400000e+02,  4.0200000e+02, ...,
           3.4300000e+02,  2.8760000e+03,  1.0000000e+00],
         ...,
         [ 4.6800000e+02,  7.4100000e+02,  8.6200000e+02, ...,
           8.5000000e+02,  1.9660000e+03,  2.0000000e+00],
         [ 5.4400000e+02,  8.1200000e+02,  9.6300000e+02, ...,
           8.1400000e+02,  2.2060000e+03,  2.0000000e+00],
         [ 5.7700000e+02,  8.2400000e+02,  9.5600000e+02, ...,
           8.6300000e+02,  2.1810000e+03,  2.0000000e+00]],

        [[ 2.1600000e+02,  3.9500000e+02,  3.8100000e+02, ...,
           3.2800000e+02,  2.8760000e+03,  1.0000000e+00],
         [ 2.1600000e+02,  3.9900000e+02,  3.9900000e+02, ...,
           3.1700000e+02,  2.9030000e+03,  1.0000000e+00],
        

In [ ]:
# def create_model(img_size,num_bands, num_classes):
#     inputs = keras.Input(shape=(img_size[0], img_size[1], num_bands))  # Change the number of channels to 4

#     # Entry block
#     x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("LeakyReLU")(x)

#     previous_block_activation = x  # Set aside residual

#     # Blocks 1, 2, 3 are identical apart from the feature depth.
#     for filters in [64, 128, 256]:
#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.SeparableConv2D(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.SeparableConv2D(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

#         # Project residual
#         residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
#             previous_block_activation
#         )
#         x = layers.add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     for filters in [256, 128, 64, 32]:
#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.UpSampling2D(2)(x)

#         residual = layers.UpSampling2D(2)(previous_block_activation)
#         residual = layers.Conv2D(filters, 1, padding="same")(residual)
#         x = layers.add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     # Add a per-pixel classification layer
#     outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

#     # Define the model
#     model = keras.Model(inputs, outputs)
#     model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # updated to sparse categorical cross-entropy loss
#     return model

In [78]:

# from tensorflow.python.keras.layers.normalization import BatchNormalization
# from tensorflow.python.keras.layers.normalization import BatchNormalization

# from tensorflow.keras.layers import BatchNormalization
# from keras.layers.normalization.batch_normalization import BatchNormalization
# from tensorflow.python.keras.layers import BatchNormalization

# U-Net model for image segmentation.
# Encoder and decoder conncted by a center block.
# Encoder downsamples the input image while capturing its features.
# Decoder upsamples the encoded image to generate a segmentation map.

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	return decoder

def get_model():
	inputs = layers.Input(shape=[KERNEL_SIZE, KERNEL_SIZE, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(CLASSES, (1, 1), activation='softmax')(decoder0) # kept because this is a multiclass classification

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [79]:
KERNEL_SIZE = 128
CLASSES = 12
BANDS = range(4)
KERNEL_SHAPE = [128, 128]
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['categorical_accuracy']

In [80]:
# import TenserFlow classes and functions
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

In [81]:
# model = create_model([128,128],4,11)
model = get_model()

In [82]:
print(model.summary()) # 31,127,148

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 4)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         1184      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 128, 32)         0         ['batch_normalization[0][0

In [ ]:
# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','categorical_accuracy'])
callbacks = [
    keras.callbacks.ModelCheckpoint("landcover_segmentation.h5", save_best_only=True)
]

model.fit(training_data_generator, epochs=10, callbacks=callbacks,shuffle=True)

Epoch 1/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 914s 1s/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 2/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 671s 1s/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 3/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 624s 948ms/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 4/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 685s 1s/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 5/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

...

658/658 [==============================] - 607s 923ms/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 7/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 581s 884ms/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 8/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 575s 874ms/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 9/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 595s 900ms/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
Epoch 10/10
658/658 [==============================] - ETA: 0s - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282

658/658 [==============================] - 540s 820ms/step - loss: nan - accuracy: 0.1282 - categorical_accuracy: 0.1282
